In [1]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'focalflow.settings.dev')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from apps.orders import  models as order_models

In [3]:
from django.db.models import Sum, Value
from django.db.models.functions import Coalesce
from datetime import datetime

# Example month filter for January 2025
month_filter = datetime(2025, 1, 1)

inventory = (
    order_models.FacturationItem.objects
    .filter(facturation__placed_at__year=month_filter.year, facturation__placed_at__month=month_filter.month)
    .values("item__id", "item__name", "item__quantity")
    .annotate(
        total_facturation=Coalesce(Sum("quantity"), Value(0))
    )
)

coverage_inventory = (
    order_models.CoverageFacturationItem.objects
    .filter(coverage_facturation__placed_at__year=month_filter.year, coverage_facturation__placed_at__month=month_filter.month)
    .values("item__id", "item__name", "item__quantity")
    .annotate(
        total_coverage_facturation=Coalesce(Sum("quantity"), Value(0))
    )
)

In [4]:
for data in inventory:
    print(data)

{'item__id': UUID('50c3e212-b982-4608-b4e9-af1b1e6ef723'), 'item__name': "Fond d'Oeil", 'item__quantity': 10, 'total_facturation': 1}
{'item__id': UUID('a3a8f8fa-5631-42b2-9f13-7dc923bf2373'), 'item__name': 'Consultation', 'item__quantity': 10, 'total_facturation': 4}


In [5]:
for data in coverage_inventory:
    print(data)

{'item__id': UUID('a3a8f8fa-5631-42b2-9f13-7dc923bf2373'), 'item__name': 'Consultation', 'item__quantity': 10, 'total_coverage_facturation': 2}


In [35]:
from collections import defaultdict

# Merge inventories
merged_inventory = defaultdict(lambda: { "total_facturation":0, "total_coverage_facturation":0, "total": 0} )

for inv in inventory :
    item_id = inv["item__id"]
    merged_inventory[item_id]["item__id"] = item_id
    merged_inventory[item_id]["item__name"] = inv["item__name"]
    merged_inventory[item_id]["item__quantity"] = inv["item__quantity"]
    merged_inventory[item_id]["total_facturation"] = inv["total_facturation"]
    merged_inventory[item_id]["total"] = inv["total_facturation"]

for inv in coverage_inventory :
    item_id = inv["item__id"]
    merged_inventory[item_id]["item__id"] = item_id
    merged_inventory[item_id]["item__name"] = inv["item__name"]
    merged_inventory[item_id]["item__quantity"] = inv["item__quantity"]
    merged_inventory[item_id]["total_coverage_facturation"] = inv["total_coverage_facturation"]
    merged_inventory[item_id]["total"] += inv["total_coverage_facturation"]

# Extend the inventory with item that has never been facturationd before
items = order_models.Item.objects.exclude(id__in=merged_inventory.keys())
for item in items:
    merged_inventory[item.id]["item__id"] = item.id
    merged_inventory[item.id]["item__name"] = item.name
    merged_inventory[item.id]["item__quantity"] = item.quantity

[{'total_facturation': 1,
  'total_coverage_facturation': 0,
  'total': 1,
  'item__id': UUID('50c3e212-b982-4608-b4e9-af1b1e6ef723'),
  'item__name': "Fond d'Oeil",
  'item__quantity': 10},
 {'total_facturation': 4,
  'total_coverage_facturation': 2,
  'total': 6,
  'item__id': UUID('a3a8f8fa-5631-42b2-9f13-7dc923bf2373'),
  'item__name': 'Consultation',
  'item__quantity': 10},
 {'total_facturation': 0,
  'total_coverage_facturation': 0,
  'total': 0,
  'item__id': UUID('2af58b65-380a-40e8-b6b5-4a14ca5cbce0'),
  'item__name': 'Opticien',
  'item__quantity': 10}]